Uma breve exploração dos dados pode set encontrado em [exploration.ipynb](./exploration.ipynb)

# Importando bibliotecas

In [1]:
%load_ext dotenv
%dotenv

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Data Extraction

In [3]:

dataset_path = os.environ['DATASET_PATH']
df = pd.read_csv(dataset_path, index_col=0)
df.head()

,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
product_id,,,,,,,,,,,,,,
11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração


# Data Formatting

In [4]:
# Save types
df['query'] = df['query'].astype('string')
df['title'] = df['title'].astype('string')
df['concatenated_tags'] = df['concatenated_tags'].astype('string')
df['concatenated_tags'][df['concatenated_tags'].isnull()] = ""
df['creation_date'] = pd.to_datetime(df['creation_date'])
df['order_counts'][df['order_counts'].isnull()] = 0
df['category'] = df['category'].astype('category')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38000 entries, 11394449 to 6866725
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   seller_id          38000 non-null  int64         
 1   query              38000 non-null  string        
 2   search_page        38000 non-null  int64         
 3   position           38000 non-null  int64         
 4   title              38000 non-null  string        
 5   concatenated_tags  38000 non-null  string        
 6   creation_date      38000 non-null  datetime64[ns]
 7   price              38000 non-null  float64       
 8   weight             37942 non-null  float64       
 9   express_delivery   38000 non-null  int64         
 10  minimum_quantity   38000 non-null  int64         
 11  view_counts        38000 non-null  int64         
 12  order_counts       38000 non-null  float64       
 13  category           38000 non-null  category      
dt

In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['category_key'] = le.fit_transform(df['category'])
df['category_key']

product_id
11394449    2
15534262    5
16153119    4
15877252    0
15917108    2
           ..
13230578    3
6736914     5
11017911    0
6807331     3
6866725     2
Name: category_key, Length: 38000, dtype: int64

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Eu não entendo direito oq isso faz, mas foi oq eu achei na net
vectorizer = TfidfVectorizer()

collum_tfid = 'concatenated_tags'
tfid = pd.DataFrame(vectorizer.fit_transform(df[collum_tfid]).toarray())
tfid.columns = [f"{collum_tfid}_{i}" for i in tfid.columns]
tfid

,concatenated_tags_0,concatenated_tags_1,concatenated_tags_2,concatenated_tags_3,concatenated_tags_4,concatenated_tags_5,concatenated_tags_6,concatenated_tags_7,concatenated_tags_8,concatenated_tags_9,...,concatenated_tags_7616,concatenated_tags_7617,concatenated_tags_7618,concatenated_tags_7619,concatenated_tags_7620,concatenated_tags_7621,concatenated_tags_7622,concatenated_tags_7623,concatenated_tags_7624,concatenated_tags_7625
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
dfinal = tfid # .join(df[['price', 'weight', 'minimum_quantity']])
dfinal.columns

Index(['concatenated_tags_0', 'concatenated_tags_1', 'concatenated_tags_2',
       'concatenated_tags_3', 'concatenated_tags_4', 'concatenated_tags_5',
       'concatenated_tags_6', 'concatenated_tags_7', 'concatenated_tags_8',
       'concatenated_tags_9',
       ...
       'concatenated_tags_7616', 'concatenated_tags_7617',
       'concatenated_tags_7618', 'concatenated_tags_7619',
       'concatenated_tags_7620', 'concatenated_tags_7621',
       'concatenated_tags_7622', 'concatenated_tags_7623',
       'concatenated_tags_7624', 'concatenated_tags_7625'],
      dtype='object', length=7626)

In [8]:
X, y = dfinal.to_numpy() , df['category'].to_numpy()
X, y

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array(['Decoração', 'Papel e Cia', 'Outros', ..., 'Bebê', 'Lembrancinhas',
        'Decoração'], dtype=object))

In [9]:
X.shape, y.shape

((38000, 7626), (38000,))

# Modeling

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=4)

In [11]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)

RandomForestClassifier()

# Validation

In [ ]:
y_pred=clf.predict(X_test)

In [19]:
from sklearn.metrics import classification_report
report = classification_report(y_pred, y_test)
print(report)

'              precision    recall  f1-score   support\n\n           0       0.84      0.89      0.87      1638\n           1       0.84      0.96      0.90       210\n           2       0.87      0.83      0.85      2257\n           3       0.94      0.87      0.91      4725\n           4       0.57      0.87      0.69       187\n           5       0.63      0.88      0.73       483\n\n    accuracy                           0.87      9500\n   macro avg       0.78      0.88      0.82      9500\nweighted avg       0.88      0.87      0.87      9500\n'

In [12]:
metrics_path = os.environ['METRICS_PATH']

with open(metrics_path, mode='w') as f:
    f.write(report)

FileNotFoundError: [Errno 2] No such file or directory: '/usr/src/data/metrics.txt'

# Model Export

In [ ]:
import pickle
metrics_path = os.environ['MODEL_PATH']

with open(model_path, mode='wb') as f:
    pickle.dump(clf, f)